# Covid-19 Data Dashboard

Welcome to the COVID-19 Data Dashboard. This interactive tool provides insights into COVID-19's impact in terms of hospital cases, beds occupied, and gender-specific statistics across Scotland, England, and Wales.

In [27]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as wdg
from uk_covid19 import Cov19API

In [28]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

# Global variable to store the data frames for each dataset
global dfs
dfs = {}


In [29]:




# Load JSON files and store the raw data in some variable.
jsondata = {}
with open("scotseries.json", "rt") as INFILE:
    jsondata['Scotland'] = json.load(INFILE)
with open("englandseries.json", "rt") as INFILE:
    jsondata['England'] = json.load(INFILE)
with open("genderdistribution.json", "rt") as INFILE:
    jsondata['WalesGender'] = json.load(INFILE)
with open("mfdistribution.json", "rt") as INFILE:
    jsondata['EnglandGender'] = json.load(INFILE)


In [30]:
# Function to parse dates in the data
def parse_date(datestring):
    return pd.to_datetime(datestring, format="%Y-%m-%d")

# Wrangle raw data into structured pandas DataFrames
def wrangle_data(rawdata):
    # Wrangle Scotland Data
    scotland_data = rawdata['Scotland']['data']
    scotland_df = pd.DataFrame(scotland_data)
    scotland_df['date'] = scotland_df['date'].apply(parse_date)

    # Wrangle England Data
    england_data = rawdata['England']['data']
    england_df = pd.DataFrame(england_data)
    england_df['date'] = england_df['date'].apply(parse_date)

    # Wrangle Wales Gender Data
    wales_gender_data = rawdata['WalesGender']['data'][0]
    wales_gender_df = pd.DataFrame(index=[x['age'] for x in wales_gender_data['males']], columns=['males', 'females', 'total'])
    for entry in wales_gender_data['males']:
        age_band = entry['age']
        male_count = entry['value']
        female_count = next(item for item in wales_gender_data['females'] if item['age'] == age_band)['value']
        wales_gender_df.loc[age_band] = [male_count, female_count, male_count + female_count]

    # Wrangle England Gender Data
    england_gender_data = rawdata['EnglandGender']['data'][0]
    england_gender_df = pd.DataFrame(index=[x['age'] for x in england_gender_data['males']], columns=['males', 'females', 'total'])
    for entry in england_gender_data['males']:
        age_band = entry['age']
        male_count = entry['value']
        female_count = next(item for item in england_gender_data['females'] if item['age'] == age_band)['value']
        england_gender_df.loc[age_band] = [male_count, female_count, male_count + female_count]

    return {
        'Scotland': scotland_df, 
        'England': england_df, 
        'WalesGender': wales_gender_df, 
        'EnglandGender': england_gender_df
    }

# Initialize the data frames with the wrangled data
dfs = wrangle_data(jsondata)


## Download current data

Stay updated with the most recent data:

Click the 'Fetch Data' button to retrieve the latest COVID-19 data.
The dashboard will update with current information from Public Health England.
Note: Data availability is subject to API connectivity and data updates from official sources.

In [31]:
# Filters and structures for API calls
# These define the parameters for API requests
scotland_filters = ['areaType=nation', 'areaName=Scotland']
england_filters = ['areaType=nation', 'areaName=England']
wales_filters = ['areaType=nation', 'areaName=Wales']
common_structure = {
    "date": "date",
    "bedsOccupied": "covidOccupiedMVBeds",
    "hospitalCases": "hospitalCases"
}
gender_structure = {
    "males": "maleCases",
    "females": "femaleCases"
}

# Function to access API and fetch new data
def access_api():
    # Define filters and structures
    scotland_filters = ['areaType=nation', 'areaName=Scotland']
    england_filters = ['areaType=nation', 'areaName=England']
    wales_filters = ['areaType=nation', 'areaName=Wales']
    common_structure = {
        "date": "date",
        "bedsOccupied": "covidOccupiedMVBeds",
        "hospitalCases": "hospitalCases"
    }
    gender_structure = {
        "males": "maleCases",
        "females": "femaleCases"
    }

    # Fetch data for each region with the appropriate structure
    scotland_api = Cov19API(filters=scotland_filters, structure=common_structure)
    england_api = Cov19API(filters=england_filters, structure=common_structure)
    wales_gender_api = Cov19API(filters=wales_filters, structure=gender_structure)
    england_gender_api = Cov19API(filters=england_filters, structure=gender_structure)

    # Get data in JSON format
    scotland_data = scotland_api.get_json()
    england_data = england_api.get_json()
    wales_gender_data = wales_gender_api.get_json()
    england_gender_data = england_gender_api.get_json()

    # Combine all data into a single dictionary
    apidata = {
        'Scotland': scotland_data,
        'England': england_data,
        'WalesGender': wales_gender_data,
        'EnglandGender': england_gender_data
    }

    return apidata


def api_button_callback(button):
    apidata = access_api()
    global dfs
    dfs = wrangle_data(apidata)
    # Refresh the graphs with new data
    refresh_line_graph()
    refresh_bar_graph()
    # Update button state
    apibutton.icon = "check"
    apibutton.description = "Data Refreshed"


# Creating the 'Fetch Data' button
apibutton = wdg.Button(
    description='Fetch Data',
    disabled=False,
    button_style='info',
    tooltip="Click to download current data",
    icon='download'
)
apibutton.on_click(api_button_callback)
display(apibutton)


Button(button_style='info', description='Fetch Data', icon='download', style=ButtonStyle(), tooltip='Click to …

## Graphs and Analysis

Explore dynamic graphs which offer insights into various aspects of the COVID-19 data:

Time-Series Analysis: Visualize trends over time, such as hospital cases and bed occupancy, for Scotland and England.

Demographic Analysis: Examine gender-specific COVID-19 case distributions in Wales and England.

Interact with the dropdown menus to switch between different datasets and gain different perspectives on the COVID-19 data.

In [32]:
# Plotting functions and widgets for data visualization
# Line Chart Plotting Function
def plot_line_chart(data_key):
    global dfs
    df = dfs[data_key]
    if data_key in ['Scotland', 'England']:
        df.plot(x='date', y=['bedsOccupied', 'hospitalCases'])
    plt.show()

# Dropdown widget for selecting the dataset for the line chart
line_data = wdg.Dropdown(
    options=['Scotland', 'England'],
    value='Scotland',
    description='Line Chart:',
    disabled=False,
)

# Function to refresh the line chart
def refresh_line_graph():
    current = line_data.value
    line_data.value = (set(line_data.options) - {current}).pop()
    line_data.value = current

# Interactive output for the line chart
line_graph = wdg.interactive_output(plot_line_chart, {'data_key': line_data})
display(line_data, line_graph)


Dropdown(description='Line Chart:', options=('Scotland', 'England'), value='Scotland')

Output()

In [26]:
# Bar Chart Plotting Function
def plot_bar_chart(data_key):
    global dfs
    df = dfs[data_key]
    if data_key in ['WalesGender', 'EnglandGender']:
        df.plot(kind='bar')
    plt.show()

# Dropdown widget for selecting the dataset for the bar chart
bar_data = wdg.Dropdown(
    options=['WalesGender', 'EnglandGender'],
    value='WalesGender',
    description='Bar Chart:',
    disabled=False,
)

# Function to refresh the bar chart
def refresh_bar_graph():
    current = bar_data.value
    bar_data.value = (set(bar_data.options) - {current}).pop()
    bar_data.value = current

# Interactive output for the bar chart
bar_graph = wdg.interactive_output(plot_bar_chart, {'data_key': bar_data})
display(bar_data, bar_graph)


Dropdown(description='Bar Chart:', options=('WalesGender', 'EnglandGender'), value='WalesGender')

Output()